## 73 SAT Wheelchair Map

### version 3.3 

* this [Notebook](https://github.com/salgo60/Stockholm_Archipelago_Trail/blob/main/notebook/Issue_73_—_SAT_Wheelchair_Map_&_Stats.ipynb)
  * Latest maps [SAT Wheel chair Map](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_WHEELCHAIR_073_wheelchair_latest.html) 

---- 
* Issue [73 SAT Wheel chair map](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/73)

* Try to get better quality WHeelchair data in OSM for SAT
---- 


       
    
  
  


In [1]:
import time

from datetime import datetime

now = datetime.now()
timestamp = now.timestamp()

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

Start: 2025-11-05 08:55:25


In [2]:
# Issue 73 — SAT: markera rullstol (wheelchair)
# ----------------------------------------------------------------------------
# Complete, notebook-ready rewrite
#
# What this does
#  - Loads SAT trail geometry (from Wikidata -> OSM Overpass)
#  - Fetches wheelchair-relevant POIs near the trail using small, reliable
#    Overpass queries (tagged first, then curated amenities/tourism), tiled if needed
#  - Computes distance bands to the trail (≤50 m, ≤100 m, ≤200 m)
#  - Builds a Folium map with points colored by wheelchair status
#      * green  = wheelchair=yes
#      * red    = wheelchair=no
#      * orange = missing/unknown
#    Layers are grouped by distance band for easy toggling.
#  - Popups start with Wheelmap link, then OSM + iD editor
#  - Exports CSVs: raw POIs and stats (status × distance band), and saves HTML
#    as timestamped + *_latest.html
#
# Dependencies (install if needed):
#   pip install requests pandas geopandas shapely folium
# ----------------------------------------------------------------------------

from __future__ import annotations

# --- Stdlib
import os
import re
import time
from pathlib import Path
from urllib.parse import quote
from datetime import datetime
import html as _html

# --- Third-party
import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString
import shapely
import folium
from folium import GeoJson, GeoJsonPopup

# =================== Config ==================================================
ISSUE_NUMBER = 73
PROJECT_NAME = "SAT_WHEELCHAIR_073"
OUTPUT_DIR   = "./output"           # change if you want another folder
STAMP        = pd.Timestamp.now().strftime("%Y%m%d_%H%M")

WD_ENDPOINT  = "https://query.wikidata.org/sparql"
OVERPASS_ENDPOINTS = [
    "https://overpass-api.de/api/interpreter",
    "https://overpass.kumi.systems/api/interpreter",
]
UA = {"User-Agent": "SAT-wheelchair/1.1 (salgo60 tools)"}

# Distance bands (meters)
DIST_BANDS = [50, 100, 200]

# Tiling the bbox keeps Overpass requests small & reliable
TILE_NX = 2
TILE_NY = 2

# Curated feature classes where wheelchair=* is most relevant
CURATED_AMENITY = (
    "toilets|cafe|restaurant|shelter|drinking_water|"
    "place_of_worship|atm|pub|bar"
)
CURATED_TOURISM = "attraction|museum|information|alpine_hut|camp_site|picnic_site|viewpoint"

# Colors
COLOR_YES = "#2ecc71"   # green
COLOR_NO  = "#e74c3c"   # red
COLOR_MIS = "#f39c12"   # orange (missing/unknown)
TRAIL_COLOR = "#111827" # near-black

# =================== Helpers: links =========================================

def _osm_view_link(osm_type: str, osmid: int | str) -> str:
    return f"https://www.openstreetmap.org/{osm_type}/{int(osmid)}"

def _osm_edit_link(osm_type: str, osmid: int | str) -> str:
    t = osm_type.lower()
    if t == "node":   return f"https://www.openstreetmap.org/edit?editor=id&node={int(osmid)}"
    if t == "way":    return f"https://www.openstreetmap.org/edit?editor=id&way={int(osmid)}"
    return f"https://www.openstreetmap.org/edit?editor=id&relation={int(osmid)}"

def _wheelmap_link(osm_type: str, osmid: int | str) -> str:
    return f"https://wheelmap.org/{osm_type}/{int(osmid)}"

# =================== Wikidata: fetch SAT sections ============================

def fetch_sat_sections_from_wikidata(trail_qid: str = "Q131318799") -> pd.DataFrame:
    query = f"""
    SELECT ?section ?sectionLabel ?osmr ?osmw_p10689 ?osmw_p11693 WHERE {{
      VALUES ?trail {{ wd:{trail_qid} }}
      {{ ?section wdt:P361+ ?trail . }} UNION {{ ?trail wdt:P527 ?section . }}
      FILTER(?section != ?trail)
      OPTIONAL {{ ?section wdt:P402 ?osmr }}
      OPTIONAL {{ ?section wdt:P10689 ?osmw_p10689 }}
      OPTIONAL {{ ?section wdt:P11693 ?osmw_p11693 }}
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "sv,en". }}
    }}
    ORDER BY ?sectionLabel
    """
    headers = {"Accept": "application/sparql-results+json", **UA}
    r = requests.get(WD_ENDPOINT, params={"query": query}, headers=headers, timeout=60)
    r.raise_for_status()
    data = r.json()
    rows = []
    for b in data["results"]["bindings"]:
        qid   = b["section"]["value"].split("/")[-1]
        label = b.get("sectionLabel", {}).get("value", qid)
        osmr  = b.get("osmr", {}).get("value")
        osmw1 = b.get("osmw_p10689", {}).get("value")
        osmw2 = b.get("osmw_p11693", {}).get("value")
        osmw  = osmw1 or osmw2
        rows.append({
            "section_qid": qid,
            "sectionLabel": label,
            "osmr": pd.to_numeric(osmr, errors="coerce"),
            "osmw": pd.to_numeric(osmw, errors="coerce"),
        })
    df = pd.DataFrame(rows)
    if not df.empty:
        df["osmr"] = df["osmr"].astype("Int64")
        df["osmw"] = df["osmw"].astype("Int64")
    return df

# =================== Overpass (robust) ======================================

def _overpass(query: str, retries: int = 2, pause: float = 2.0) -> dict:
    headers = {
        "User-Agent": UA["User-Agent"],
        "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
        "Accept": "application/json",
    }
    last_err = None
    for ep in OVERPASS_ENDPOINTS:
        for i in range(retries):
            try:
                resp = requests.post(ep, data={"data": query}, headers=headers, timeout=180)
                if resp.status_code == 429:
                    time.sleep(pause * (i + 1)); continue
                if resp.status_code >= 400:
                    raise RuntimeError(
                        f"Overpass HTTP {resp.status_code} @ {ep}\n--- Query ---\n{query}\n--- Server says ---\n{resp.text[:2000]}\n"
                    )
                return resp.json()
            except Exception as e:
                last_err = e
                time.sleep(pause * (i + 1))
    raise RuntimeError(f"Overpass failed on all endpoints.\nLast error:\n{last_err}")

# =================== SAT ways from OSM ======================================

def _fetch_ways_for_relation(rel_id: int) -> list[dict]:
    q = f"""
[out:json][timeout:180];
relation({rel_id});
way(r);
out tags geom;
"""
    data = _overpass(q)
    return [el for el in data.get("elements", []) if el.get("type") == "way"]


def _fetch_way(way_id: int) -> list[dict]:
    q = f"""
[out:json][timeout:120];
way({way_id});
out tags geom;
"""
    data = _overpass(q)
    return [el for el in data.get("elements", []) if el.get("type") == "way"]

CORE_WAY_TAGS = ["name","highway","surface","tracktype","foot","bicycle","sac_scale","trail_visibility","smoothness","width","image"]

def _way_to_feature(el: dict) -> dict | None:
    if "geometry" not in el: return None
    coords = [(pt["lon"], pt["lat"]) for pt in el["geometry"]]
    if len(coords) < 2: return None
    tags = el.get("tags", {}) or {}
    props = {"osmid": el["id"]}
    for k in CORE_WAY_TAGS: props[k] = tags.get(k)
    return {"properties": props, "geometry": LineString(coords)}


def _elements_to_gdf(elements: list[dict]) -> gpd.GeoDataFrame:
    feats = []
    for el in elements:
        f = _way_to_feature(el)
        if f: feats.append(f)
    if not feats:
        return gpd.GeoDataFrame(columns=["osmid",*CORE_WAY_TAGS,"geometry"], geometry="geometry", crs="EPSG:4326")
    df  = pd.DataFrame([f["properties"] for f in feats])
    gdf = gpd.GeoDataFrame(df, geometry=[f["geometry"] for f in feats], crs="EPSG:4326")
    for k in ["osmid",*CORE_WAY_TAGS]:
        if k not in gdf.columns: gdf[k] = None
    return gdf


def build_sat_seg_gdfs(trail_qid: str = "Q131318799", prefer_relation: bool = True) -> dict[str, gpd.GeoDataFrame]:
    meta = fetch_sat_sections_from_wikidata(trail_qid)
    if meta.empty:
        print("Hittade inga etapper i Wikidata för", trail_qid)
        return {}
    out: dict[str, gpd.GeoDataFrame] = {}
    for _, row in meta.iterrows():
        label  = str(row["sectionLabel"]).strip() or row["section_qid"]
        rel_id = row["osmr"] if pd.notna(row["osmr"]) else None
        way_id = row["osmw"] if pd.notna(row["osmw"]) else None
        elements: list[dict] = []
        if prefer_relation and rel_id:
            try:
                elements = _fetch_ways_for_relation(int(rel_id))
            except Exception as e:
                print(f"[{label}] relation {rel_id} failed: {e}")
        if (not elements) and way_id:
            try:
                elements = _fetch_way(int(way_id))
            except Exception as e:
                print(f"[{label}] way {way_id} failed: {e}")
        gdf = _elements_to_gdf(elements)
        if "name" not in gdf.columns:
            gdf["name"] = None
        gdf["name"] = gdf["name"].fillna(label)
        out[label] = gdf
    return out

# =================== BBox helpers & tiling ==================================

def _bbox_from_gdfs(gdfs: dict[str, gpd.GeoDataFrame]) -> tuple[float,float,float,float]:
    boxes = []
    for g in gdfs.values():
        if g is None or g.empty: continue
        minx, miny, maxx, maxy = g.to_crs(epsg=4326).total_bounds
        boxes.append((minx, miny, maxx, maxy))
    if not boxes:
        return (18.0, 59.0, 19.0, 60.0)
    minx = min(b[0] for b in boxes); miny = min(b[1] for b in boxes)
    maxx = max(b[2] for b in boxes); maxy = max(b[3] for b in boxes)
    return (minx, miny, maxx, maxy)


def _expand_bbox_meters(bbox4326: tuple[float,float,float,float], meters: int = 1500) -> tuple[float,float,float,float]:
    gs = gpd.GeoSeries([shapely.geometry.box(*bbox4326)], crs="EPSG:4326").to_crs(epsg=3857)
    minx, miny, maxx, maxy = gs.total_bounds
    minx -= meters; miny -= meters; maxx += meters; maxy += meters
    bb2 = gpd.GeoSeries([shapely.geometry.box(minx, miny, maxx, maxy)], crs="EPSG:3857").to_crs(epsg=4326)
    return tuple(bb2.total_bounds.tolist())  # type: ignore


def _tile_bbox(minx, miny, maxx, maxy, nx=TILE_NX, ny=TILE_NY):
    dx = (maxx - minx) / max(nx,1)
    dy = (maxy - miny) / max(ny,1)
    tiles = []
    for ix in range(nx):
        for iy in range(ny):
            x0 = minx + ix*dx; x1 = x0 + dx
            y0 = miny + iy*dy; y1 = y0 + dy
            tiles.append((x0, y0, x1, y1))
    return tiles

# =================== Overpass queries for POIs ===============================

def _q_tagged_wheelchair(minx, miny, maxx, maxy) -> str:
    # only objects that already have wheelchair=*
    return f"""
[out:json][timeout:120];
(
  node["wheelchair"]({miny},{minx},{maxy},{maxx});
  way ["wheelchair"]({miny},{minx},{maxy},{maxx});
);
out body tags center qt;
"""


def _q_curated_candidates(minx, miny, maxx, maxy) -> str:
    # likely relevant objects even if wheelchair is missing
    return f"""
[out:json][timeout:120];
(
  node["amenity"~"^({CURATED_AMENITY})$"]({miny},{minx},{maxy},{maxx});
  way ["amenity"~"^({CURATED_AMENITY})$"]({miny},{minx},{maxy},{maxx});
  node["tourism"~"^({CURATED_TOURISM})$"]({miny},{minx},{maxy},{maxx});
  way ["tourism"~"^({CURATED_TOURISM})$"]({miny},{minx},{maxy},{maxx});
);
out body tags center qt;
"""


def fetch_wheelchair_candidates(minx: float, miny: float, maxx: float, maxy: float, tiled: bool = True) -> dict:
    """Two-stage fetch (tagged first, then curated), optionally tiled."""
    tiles = _tile_bbox(minx, miny, maxx, maxy) if tiled else [(minx, miny, maxx, maxy)]
    elements = []
    for (x0, y0, x1, y1) in tiles:
        for q in (_q_tagged_wheelchair(x0, y0, x1, y1), _q_curated_candidates(x0, y0, x1, y1)):
            part = _overpass(q)
            if part and "elements" in part:
                elements.extend(part["elements"])
        time.sleep(0.5)  # be nice to Overpass
    return {"elements": elements}

# =================== Convert POIs to GeoDataFrame ===========================

def _pois_to_points_gdf(data: dict) -> gpd.GeoDataFrame:
    rows = []
    for el in data.get("elements", []):
        t = el.get("type")
        tags = el.get("tags", {}) or {}
        if t == "node":
            lon = el.get("lon"); lat = el.get("lat")
            if lon is None or lat is None: continue
            rows.append({
                "osm_type": "node",
                "osmid": el["id"],
                "lon": lon, "lat": lat,
                "name": tags.get("name"),
                "wheelchair": tags.get("wheelchair"),
                "amenity": tags.get("amenity"),
                "tourism": tags.get("tourism"),
            })
        elif t == "way":
            c = el.get("center")
            if not c: continue  # we used 'out center'; skip if missing
            rows.append({
                "osm_type": "way",
                "osmid": el["id"],
                "lon": c.get("lon"), "lat": c.get("lat"),
                "name": tags.get("name"),
                "wheelchair": tags.get("wheelchair"),
                "amenity": tags.get("amenity"),
                "tourism": tags.get("tourism"),
            })
    if not rows:
        return gpd.GeoDataFrame(columns=["osm_type","osmid","name","wheelchair","amenity","tourism","geometry"], geometry="geometry", crs="EPSG:4326")
    df = pd.DataFrame(rows).dropna(subset=["lon","lat"])  # safety
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df["lon"], df["lat"]), crs="EPSG:4326")
    return gdf

# =================== Distance & bands =======================================

def _nearest_distance_to_lines(point_m: shapely.geometry.base.BaseGeometry, lines_union_m: shapely.geometry.base.BaseGeometry) -> float:
    if lines_union_m.is_empty: return float("inf")
    return float(point_m.distance(lines_union_m))


def _band_label(d_m: float) -> str:
    for thr in DIST_BANDS:
        if d_m <= thr: return f"≤{thr}m"
    return f">{DIST_BANDS[-1]}m"

# =================== Popup builder ==========================================



def _popup_html(row: pd.Series) -> str:
    osm_type = str(row.get("osm_type","way"))
    osmid    = int(row.get("osmid")) if pd.notna(row.get("osmid")) else None
    title    = row.get("name") or f"{osm_type} {osmid}"
    status   = (str(row.get("wheelchair")) or "").strip().lower()
    status_h = status if status in {"yes","no","limited","designated"} else ("unknown" if status else "missing")
    color    = COLOR_YES if status == "yes" else (COLOR_NO if status == "no" else COLOR_MIS)

    wl = _wheelmap_link(osm_type, osmid) if osmid else "#"
    ov = _osm_view_link(osm_type, osmid) if osmid else "#"
    oe = _osm_edit_link(osm_type, osmid) if osmid else "#"

    meta = []
    for k in ("amenity","tourism"):
        if pd.notna(row.get(k)):
            meta.append(f"{k}={_html.escape(str(row[k]))}")

    dist_info = f"Dist to trail: {row.get('dist_m', float('nan')):.0f} m ({row.get('dist_band','')})"

    # Build the optional metadata chip separately (prevents nested f-string issues)
    meta_chip = ""
    if meta:
        joined = " • ".join(meta)
        meta_chip = (
            "<span style=\"display:inline-block;padding:2px 8px;border-radius:999px;"
            "background:#6e7781;color:white;\">" + joined + "</span>"
        )

    html = f"""
<div style="max-width:520px">
  <div style="border-radius:12px;padding:10px;background:#ffffff;box-shadow:0 1px 6px rgba(0,0,0,.08);font-family:system-ui,-apple-system,Segoe UI,Roboto,Ubuntu,Helvetica,Arial,sans-serif;font-size:14px;color:#24292f;">
    <div style="font-weight:700;font-size:18px;margin:0 0 6px 0;">{_html.escape(str(title))}</div>
    <div style="margin:6px 0;display:flex;gap:8px;align-items:center;">
      <a href="{_html.escape(wl)}" target="_blank" style="display:inline-block;padding:4px 8px;border-radius:999px;background:#1f6feb;color:white;text-decoration:none;">Wheelmap</a>
      <a href="{_html.escape(ov)}" target="_blank" style="display:inline-block;padding:4px 8px;border-radius:999px;background:#374151;color:white;text-decoration:none;">OSM</a>
      <a href="{_html.escape(oe)}" target="_blank" style="display:inline-block;padding:4px 8px;border-radius:999px;background:#111827;color:white;text-decoration:none;">iD editor</a>
    </div>
    <div style="margin:6px 0;display:flex;flex-wrap:wrap;gap:6px;align-items:center;">
      <span style="display:inline-block;padding:2px 8px;border-radius:999px;background:{color};color:white;">wheelchair: {status_h}</span>
      <span style="display:inline-block;padding:2px 8px;border-radius:999px;background:#6e7781;color:white;">{_html.escape(dist_info)}</span>
      {meta_chip}
    </div>
  </div>
</div>
"""
    return html

# =================== Map & stats ============================================

def make_wheelchair_map(etapp_gdfs: dict[str, gpd.GeoDataFrame],
                        OUTPUT_DIR: str = OUTPUT_DIR,
                        PROJECT_NAME: str = PROJECT_NAME,
                        STAMP: str = STAMP,
                        tiled_fetch: bool = True) -> tuple[str, pd.DataFrame]:
    if not etapp_gdfs:
        raise RuntimeError("No SAT data — build etapp_gdfs first")

    # Combine SAT lines
    lines = []
    for section, g in etapp_gdfs.items():
        if g is None or g.empty: continue
        gf = g[["geometry"]].copy(); gf["section"] = section
        lines.append(gf)
    sat = gpd.GeoDataFrame(pd.concat(lines, ignore_index=True), geometry="geometry", crs="EPSG:4326")

    # Bbox and fetch POIs
    minx, miny, maxx, maxy = _bbox_from_gdfs(etapp_gdfs)
    bbox_exp = _expand_bbox_meters((minx, miny, maxx, maxy), meters=1500)
    data = fetch_wheelchair_candidates(*bbox_exp, tiled=tiled_fetch)
    pois = _pois_to_points_gdf(data)

    # Prepare map
    minx, miny, maxx, maxy = sat.total_bounds
    m = folium.Map(location=[(miny+maxy)/2, (minx+maxx)/2], zoom_start=10, control_scale=True, tiles="OpenStreetMap")
    GeoJson(sat[["geometry"]], name="SAT", style_function=lambda _f: {"weight": 4, "color": TRAIL_COLOR}).add_to(m)

    if pois.empty:
        folium.LayerControl(collapsed=False).add_to(m)
        os.makedirs(OUTPUT_DIR, exist_ok=True)
        html_path = os.path.join(OUTPUT_DIR, f"{PROJECT_NAME}_wheelchair_{STAMP}.html")
        _add_about_box(m, issue_number=ISSUE_NUMBER, map_name="Wheelchair near SAT")
        _save_with_latest(m, html_path)
        return html_path, pd.DataFrame()

    # Distances
    sat_m  = sat.to_crs(epsg=3857)
    pois_m = pois.to_crs(epsg=3857)
    lines_union = sat_m.unary_union
    pois_m["dist_m"] = pois_m.geometry.apply(lambda g: _nearest_distance_to_lines(g, lines_union))
    pois_m["dist_band"] = pois_m["dist_m"].apply(_band_label)

    # Keep ≤200 m
    pois_m = pois_m.loc[pois_m["dist_m"] <= DIST_BANDS[-1]].copy()

    # Back to WGS84
    pois_4326 = pois_m.to_crs(epsg=4326)

    # Color by status
    def _status_color(s):
        s = (str(s) if s is not None else "").strip().lower()
        if s == "yes": return COLOR_YES
        if s == "no":  return COLOR_NO
        return COLOR_MIS

    # Layers per distance band
    for bl in [f"≤{DIST_BANDS[0]}m", f"≤{DIST_BANDS[1]}m", f"≤{DIST_BANDS[2]}m"]:
        sel = pois_4326.loc[pois_4326["dist_band"] == bl]
        if sel.empty: continue
        fg = folium.FeatureGroup(name=f"POIs {bl}")
        for _, r in sel.iterrows():
            color = _status_color(r.get("wheelchair"))
            folium.CircleMarker(
                location=[r.geometry.y, r.geometry.x], radius=6, weight=2, color=color, fill=True, fill_opacity=0.9,
                popup=folium.Popup(_popup_html(r), max_width=520)
            ).add_to(fg)
        fg.add_to(m)

    # Legend
    legend_html = f"""
    <div style=\"position: fixed; bottom: 18px; left: 18px; z-index: 9999; background: rgba(255,255,255,0.92);\
                padding: 10px 12px; border-radius: 10px; box-shadow: 0 2px 8px rgba(0,0,0,0.15); font-size: 13px;\">
      <div style=\"font-weight:600; margin-bottom:6px;\">Wheelchair status</div>
      <div><span style=\"display:inline-block;width:14px;height:3px;background:{COLOR_YES};margin-right:6px;\"></span>yes (grön)</div>
      <div><span style=\"display:inline-block;width:14px;height:3px;background:{COLOR_NO};margin-right:6px;\"></span>no (röd)</div>
      <div><span style=\"display:inline-block;width:14px;height:3px;background:{COLOR_MIS};margin-right:6px;\"></span>saknas/okänd (orange)</div>
    </div>
    """
    m.get_root().html.add_child(folium.Element(legend_html))

    folium.LayerControl(collapsed=False).add_to(m)
    _add_about_box(m, issue_number=ISSUE_NUMBER, map_name="Wheelchair near SAT (≤200 m)")

    # Save
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    html_path = os.path.join(OUTPUT_DIR, f"{PROJECT_NAME}_wheelchair_{STAMP}.html")
    _save_with_latest(m, html_path)

    # Stats (status_bucket × distance band)
    df_stats = (pois_4326
        .assign(status=lambda d: d["wheelchair"].fillna("").str.lower().replace({"nan":""}))
        .assign(status_bucket=lambda d: d["status"].where(d["status"].isin(["yes","no"]), other="missing"))
        .groupby(["status_bucket","dist_band"]).size().reset_index(name="count")
    )

    # Per-section stats
    pois_sections = gpd.sjoin_nearest(
        pois_4326,
        sat[["section","geometry"]].to_crs(epsg=4326),
        how="left",
        distance_col="nearest_dist"
    )
    #pois_sections = gpd.sjoin(pois_4326, sat[["section","geometry"]].to_crs(epsg=4326), how="left", predicate="nearest")
    df_stats_section = (pois_sections
    .assign(status=lambda d: d["wheelchair"].fillna("").str.lower().replace({"nan":""}))
    .assign(status_bucket=lambda d: d["status"].where(d["status"].isin(["yes","no"]), other="missing"))
    .groupby(["section","status_bucket","dist_band"]).size().reset_index(name="count")
    )
    
    
    # Also write CSV with raw POIs
    raw_csv = os.path.join(OUTPUT_DIR, f"{PROJECT_NAME}_wheelchair_raw_{STAMP}.csv")
    pois_4326.drop(columns=["geometry"]).to_csv(raw_csv, index=False)
    
    
    stats_csv = os.path.join(OUTPUT_DIR, f"{PROJECT_NAME}_wheelchair_stats_{STAMP}.csv")
    df_stats.to_csv(stats_csv, index=False)
    
    
    stats_section_csv = os.path.join(OUTPUT_DIR, f"{PROJECT_NAME}_wheelchair_stats_sections_{STAMP}.csv")
    df_stats_section.to_csv(stats_section_csv, index=False)
    
    
    print("Saved map:", html_path)
    print("Saved stats:", stats_csv)
    print("Saved per-section stats:", stats_section_csv)
    print("Saved raw POIs:", raw_csv)
    # ==== Per-section stats (nearest section) ===================================
    try:
        sat_sections = sat[["section", "geometry"]].copy()
        pois_with_sec = gpd.sjoin_nearest(pois_4326, sat_sections, how="left")
        df_stats_section = (
            pois_with_sec.assign(
                status=lambda d: d["wheelchair"].fillna("").str.lower().replace({"nan": ""}),
                status_bucket=lambda d: d["status"].where(d["status"].isin(["yes","no"]), other="missing"),
            )
            .groupby(["section","status_bucket","dist_band"]).size()
            .reset_index(name="count")
        )
        stats_section_csv = os.path.join(OUTPUT_DIR, f"{PROJECT_NAME}_wheelchair_stats_sections_{STAMP}.csv")
        df_stats_section.to_csv(stats_section_csv, index=False)
        print("Saved per-section stats:", stats_section_csv)
    except Exception as e:
        print("Per-section stats skipped:", e)

    return html_path, df_stats_section
# =================== About box & save helpers ===============================

from string import Template

def _add_about_box(
    m,
    issue_number: int,
    map_name: str,
    created_date: str | None = None,
    repo: str = "salgo60/Stockholm_Archipelago_Trail",
    collapsed: bool = False,
):
    if created_date is None:
        created_date = datetime.now().strftime("%Y-%m-%d %H:%M")

    map_dom_id = m.get_name()
    box_id     = f"sat-about-{map_dom_id}"
    header_id  = f"{box_id}-hdr"
    issue_url  = f"https://github.com/{repo}/issues/{issue_number}"

    links = [
        ("SAT Dashboard","dashboard.html"),
        ("Project repo issues", "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues?q=is%3Aissue"),
        ("Wheelmap", "https://wheelmap.org/"),
        ("Trail on OSM (rel 19012437)", "https://www.openstreetmap.org/relation/19012437"),
    ]
    links_html = "".join(
        f'<div><a href="{_html.escape(u)}" target="_blank" style="text-decoration:none;">🔗 {_html.escape(t)}</a></div>'
        for t, u in links
    )
    collapsed_class = "sat-about-collapsed" if collapsed else ""

    tpl = Template(r"""
<style>
  .sat-about { position: fixed; z-index: 10000; background: rgba(255,255,255,0.97);
    border: 2px solid #666; border-radius: 10px; box-shadow: 0 2px 6px rgba(0,0,0,0.25);
    font: 12px/1.35 -apple-system, system-ui, Segoe UI, Roboto, Helvetica, Arial, sans-serif;
    pointer-events: auto; min-width: 240px; max-width: 320px; }
  .sat-about-header { cursor: pointer; padding: 8px 10px; font-weight: 700;
    display: flex; align-items: center; gap: 6px; user-select: none;
    background: rgba(248,248,248,.9); border-bottom: 1px solid #e5e7eb; }
  .sat-about-body { padding: 8px 10px 10px 10px; }
  .sat-about-collapsed .sat-about-body { display: none; }
  .sat-chevron { margin-left: auto; transition: transform .15s ease-in-out; }
  .sat-about-collapsed .sat-chevron { transform: rotate(-90deg); }
  .sat-links { margin-top: 6px; padding-top: 6px; border-top: 1px solid #e5e7eb; }
</style>
<div id="$box_id" class="sat-about $collapsed_class">
  <div id="$header_id" class="sat-about-header" title="Click to collapse/expand">
    <span>ℹ️ About</span>
    <span class="sat-chevron">▸</span>
  </div>
  <div class="sat-about-body">
    <div style="font-weight:700;margin-bottom:4px;">SAT Wheelchair Map</div>
    <div>Issue: <a href="$issue_url" target="_blank">#$issue_number</a>&nbsp;&nbsp; Map: $map_name</div>
    <div>Created: $created_date</div>
    <div class="sat-links">$links_html</div>
  </div>
</div>
<script>
(function(){
  var mapId = "$map_dom_id";
  var boxId = "$box_id";
  var hdrId = "$header_id";
  var storageKey = "satAboutCollapsed_" + "$map_dom_id" + "_#$issue_number";
  function setCollapsed(box, collapsed) {
    var body = box.querySelector(".sat-about-body");
    if (collapsed) { box.classList.add("sat-about-collapsed"); if (body) body.style.display = "none"; }
    else { box.classList.remove("sat-about-collapsed"); if (body) body.style.display = "block"; }
    try { localStorage.setItem(storageKey, collapsed ? "1" : "0"); } catch(e) {}
  }
  function placeBox(mapEl, box) {
    var zoom = mapEl.querySelector(".leaflet-control-zoom");
    var mr   = mapEl.getBoundingClientRect();
    var top = 10, left = 10;
    if (zoom) {
      var zr = zoom.getBoundingClientRect();
      top  = (zr.bottom - mr.top) + 8;
      left = (zr.left   - mr.left) + zr.width + 8;
    }
    box.style.top  = top  + "px";
    box.style.left = left + "px";
  }
  function init(tries) {
    tries = tries || 0;
    var mapEl = document.getElementById(mapId);
    var box   = document.getElementById(boxId);
    var hdr   = document.getElementById(hdrId);
    if (!mapEl || !box || !hdr) { if (tries < 60) return setTimeout(function(){ init(tries+1); }, 120); return; }
    try {
      var stored = localStorage.getItem(storageKey);
      if (stored === "1") setCollapsed(box, true);
      if (stored === "0") setCollapsed(box, false);
    } catch(e) {}
    hdr.addEventListener("click", function(e){ e.stopPropagation(); setCollapsed(box, !box.classList.contains("sat-about-collapsed")); });
    function doPlace(){ placeBox(mapEl, box); }
    var placeTries = 0, iv = setInterval(function(){ doPlace(); if (++placeTries > 25) clearInterval(iv); }, 120);
    window.addEventListener("resize", doPlace);
    requestAnimationFrame(doPlace);
  }
  if (document.readyState === "loading") { document.addEventListener("DOMContentLoaded", function(){ init(0); }); }
  else { init(0); }
})();
</script>
""")

    html_snippet = tpl.substitute(
        box_id=box_id,
        header_id=header_id,
        collapsed_class=collapsed_class,
        issue_url=issue_url,
        issue_number=str(issue_number),
        map_name=_html.escape(map_name),
        created_date=_html.escape(created_date),
        links_html=links_html,
        map_dom_id=map_dom_id,
    )
    m.get_root().html.add_child(folium.Element(html_snippet))



def _to_latest_name(filename: str | Path) -> Path:
    p = Path(filename)
    stem = p.stem
    new_stem = re.sub(r"_(?:20\d{6})(?:_\d{4})?$", "", stem)
    return p.with_name(new_stem + "_latest.html")


def _save_with_latest(map_object, filename: str | Path):
    map_object.save(str(filename))
    latest_path = _to_latest_name(filename)
    Path(filename).parent.mkdir(parents=True, exist_ok=True)
    from shutil import copyfile
    copyfile(filename, latest_path)
    print(f"Saved: {filename}\nUpdated: {latest_path}")

# =================== Run (single cell in notebook) ==========================
if __name__ == "__main__":
    # 1) Build SAT sections
    etapp_gdfs = build_sat_seg_gdfs(trail_qid="Q131318799", prefer_relation=True)

    # 2) Build wheelchair map & stats
    html_path, stats_df = make_wheelchair_map(
        etapp_gdfs,
        OUTPUT_DIR=OUTPUT_DIR,
        PROJECT_NAME=PROJECT_NAME,
        STAMP=STAMP,
        tiled_fetch=True,
    )

    # 3) Quick console preview of stats
    if not stats_df.empty:
        try:
            print(stats_df.sort_values(["status_bucket","dist_band"]).to_string(index=False))
        except Exception:
            print(stats_df.head())

    print("✅ Wheelchair analysis ready →", html_path)


/var/folders/fd/md6r13sj0wsbg_6_xl160d300000gn/T/ipykernel_87094/614333921.py:444: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  lines_union = sat_m.unary_union


Saved: ./output/SAT_WHEELCHAIR_073_wheelchair_20251105_0855.html
Updated: output/SAT_WHEELCHAIR_073_wheelchair_latest.html
Saved map: ./output/SAT_WHEELCHAIR_073_wheelchair_20251105_0855.html
Saved stats: ./output/SAT_WHEELCHAIR_073_wheelchair_stats_20251105_0855.csv
Saved per-section stats: ./output/SAT_WHEELCHAIR_073_wheelchair_stats_sections_20251105_0855.csv
Saved raw POIs: ./output/SAT_WHEELCHAIR_073_wheelchair_raw_20251105_0855.csv
Saved per-section stats: ./output/SAT_WHEELCHAIR_073_wheelchair_stats_sections_20251105_0855.csv
      section status_bucket dist_band  count
  SAT Arholma       missing     ≤100m      3
 SAT Finnhamn       missing     ≤100m      8
SAT Fjärdlång       missing     ≤100m      1
 SAT Furusund       missing     ≤100m      3
   SAT Grinda       missing     ≤100m      7
 SAT Landsort       missing     ≤100m      1
     SAT Lidö       missing     ≤100m      5
     SAT Möja       missing     ≤100m      4
  SAT Nåttarö       missing     ≤100m      1
     SAT Or

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/geopandas/array.py:408: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/geopandas/array.py:408: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [3]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time# Bygg audit-lager för den här etappen

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("Total time elapsed: {:.2f} seconds".format(elapsed_time))



Date: 2025-11-05 08:58:23
Total time elapsed: 177.98 seconds
